In [ ]:
######## load topography and wind vector data #########

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr


import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Reshape, Conv1D, Conv2D, Flatten, Input, Normalization
from keras.optimizers import Adam, SGD, RMSprop



In [ ]:
#import h5

In [ ]:
# Specify directory for data
drct='/glade/p/cesm/amwg_dev/juliob/SAMwrf/Curtains/'
# Get data (v7 has corrected ANGLL)
tag='ndg04'
fi=drct+'SAMwrf_'+tag+'_ML_super_v7.nc'
ds=xr.open_dataset( fi )



In [ ]:
mxd = ds['MXDIS']
ang = ds['ANGLL']
cln = ds['CLNGT']
ani = ds['ANISO']
sgh = ds['SGH']

u=ds['U']
v=ds['V']
tu = ds['UTEND_NDG']*86400
tv = ds['VTEND_NDG']*86400

In [ ]:
mxdv = mxd.values
angv = ang.values
clnv = cln.values
aniv = ani.values
sghv = sgh.values

nT,nL,nS = np.shape(u)

In [ ]:
ii=np.where(mxd[0,:]>50.)
print(np.shape(ii))

mxds=mxdv[0,ii[0][:]]
clns=clnv[0,ii[0][:]]
angs=angv[0,ii[0][:]]
anis=aniv[0,ii[0][:]]
sghs=sghv[ii[0]]

In [ ]:
u=u.values
u=u[:,21:,ii[0][:]]
u=np.transpose(u, (1,0,2) )
nL,nT,nS=np.shape(u)
u=np.reshape( u, (nL,nT*nS ) )  ### should normalize each individual variables

v=v.values
v=v[:,21:,ii[0][:]]
v=np.transpose(v, (1,0,2) )
v=np.reshape( v, (nL,nT*nS ) )### should normalize each individual variables

tu = tu.values
tu=tu[:,21:,ii[0][:]]
tu=np.transpose(tu, (1,0,2) )
tu=np.reshape( tu, (nL,nT*nS ) )### should normalize each individual variables

tv = tv.values
tv=tv[:,21:,ii[0][:]]
tv=np.transpose(tv, (1,0,2) )
tv=np.reshape( tv, (nL,nT*nS ) )### should normalize each individual variables

In [ ]:
angx=np.zeros([1,nT,nS])
mxdx=np.zeros([1,nT,nS])
clnx=np.zeros([1,nT,nS])
anix=np.zeros([1,nT,nS])
sghx=np.zeros([1,nT,nS])


for iT in np.arange(nT):
    angx[0,iT,:]=angs
    mxdx[0,iT,:]=mxds
    clnx[0,iT,:]=clns
    anix[0,iT,:]=anis
    sghx[0,iT,:]=sghs
    
angx = np.reshape( angx, (1,nT*nS ) )
mxdx = np.reshape( mxdx, (1,nT*nS ) )
clnx = np.reshape( clnx, (1,nT*nS ) )
anix = np.reshape( anix, (1,nT*nS ) )
sghx = np.reshape( sghx, (1,nT*nS ) )

In [ ]:
cu = u
cu[np.where(np.isnan(cu))]=np.nanmean(cu) ### remove nan values

cv = v
cv[np.where(np.isnan(cv))]=np.nanmean(cv) ### remove nan values

ctu = tu
ctu[np.where(np.isnan(ctu))]=np.nanmean(ctu) 

ctv = tv
ctv[np.where(np.isnan(ctv))]=np.nanmean(ctv) 

In [ ]:
x_total = np.r_[cu,cv,angx,mxdx,clnx,anix,sghx]
y_total = np.r_[ctu,ctv]
x_total = np.transpose(x_total)
y_total = np.transpose(y_total)


In [ ]:
##### After data preprocessing, import tensorflow and keras build the CNN regression model, train and test ###########
normalizer = Normalization(axis=-1)
normalizer.adapt(x_total)
print(normalizer.mean.numpy())


In [ ]:
na = np.arange(0.1,0.99,0.01)
score_train = np.zeros(89)
score_eval = np.zeros(89)
kk = 0

for nn in na:

    kk=kk+1
    x_train, x_test, y_train, y_test = train_test_split(x_total, y_total, test_size=nn, random_state=40) ## split training and test dataset
    n_inputs = 27
    n_outputs = 22
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu')) ### MLP
    model.add(Dense(n_outputs))
    model.compile(loss='mae', optimizer='adam')
#add model layers
#model.compile(optimizer='Adam',
#              loss='mse'), ##### 'mean_absolute_error')


    model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))

    score = model.evaluate(x_test, y_test, verbose=1)
    y_pred = model.predict(x_test)
    score2 = model.evaluate(x_train, y_train)
    
##    score_train[kk]=score
##    score_eval[kk]=score2

In [ ]:
from sklearn.preprocessing import normalize

In [ ]:
### open data files ########

##fn ='/glade/u/home/ytian/ML_data/SAMwrf_ndg05_ML_super.nc'
fn ='/glade/u/home/ytian/ML_data/SAMwrf_ndg04_ML_2010-06.nc'
##ds = xr.open_dataset(fn)

##drct='/glade/p/cesm/amwg_dev/juliob/SAMwrf/Curtains/'
# Get data (v7 has corrected ANGLL)
##tag='ndg04'
##fn=drct+'SAMwrf_'+tag+'_ML_super_v7.nc '
ds=xr.open_dataset( fn )

In [ ]:
print(list(ds.variables))

In [ ]:
print(np.shape(ds['ANGLL']))

In [ ]:
mxd = ds['MXDIS']
ang = ds['ANGLL']
cln = ds['CLNGT']
ani = ds['ANISO']
sgh = ds['SGH']

u=ds['U']
v=ds['V']
tu = ds['UTEND_NDG']*86400
tv = ds['VTEND_NDG']*86400

In [ ]:
print(mxd.shape)

In [ ]:
print(ang.shape)

In [ ]:
print(cln.shape)

In [ ]:
print(sgh.shape)


In [ ]:
print(u.shape)

In [ ]:
print(tu.shape)

In [ ]:
mxdv = mxd.values
angv = ang.values
clnv = cln.values
aniv = ani.values
sghv = sgh.values

nT,nL,nS = np.shape(u)
##plt.plot(tu[100,10,:])

plt.plot(sgh.values.flatten(),'.')

In [ ]:
print(np.shape(mxdv))

In [ ]:
jj=np.where(np.isnan(u))
print(jj)

In [ ]:
u.shape
uc = u[~np.isnan(u)]

In [ ]:
u.shape

In [ ]:
ii=np.where(mxd[0,:]>50.)
print(np.shape(ii))

mxds=mxdv[0,ii[0][:]]
clns=clnv[0,ii[0][:]]
angs=angv[0,ii[0][:]]
anis=aniv[0,ii[0][:]]
sghs=sghv[ii[0]]

fig=plt.figure( figsize =(20,9))
ax=fig.add_subplot(2,2,1)
po=ax.plot( mxds, '.')
po=ax.set_title( 'MXDIS' ,fontsize=12, loc='center')
ax=fig.add_subplot(2,2,2)
po=ax.plot( clns, '.')
po=ax.set_title( 'CLNGT' ,fontsize=12, loc='center')
ax=fig.add_subplot(2,2,3)
po=ax.plot( angs, '.')
po=ax.set_title( 'Cos(ANGLL)' ,fontsize=12, loc='center')
ax=fig.add_subplot(2,2,4)
po=ax.plot( anis, '.')
po=ax.set_title( 'Sin (ANGLL)' ,fontsize=12, loc='center')


In [ ]:
print(np.shape(mxds))

In [ ]:
u=u.values
u=u[:,21:,ii[0][:]]
u=np.transpose(u, (1,0,2) )
nL,nT,nS=np.shape(u)
u=np.reshape( u, (nL,nT*nS ) )  ### should normalize each individual variables

v=v.values
v=v[:,21:,ii[0][:]]
v=np.transpose(v, (1,0,2) )
v=np.reshape( v, (nL,nT*nS ) )### should normalize each individual variables

tu = tu.values
tu=tu[:,21:,ii[0][:]]
tu=np.transpose(tu, (1,0,2) )
tu=np.reshape( tu, (nL,nT*nS ) )### should normalize each individual variables

tv = tv.values
tv=tv[:,21:,ii[0][:]]
tv=np.transpose(tv, (1,0,2) )
tv=np.reshape( tv, (nL,nT*nS ) )### should normalize each individual variables

In [ ]:
print(nS)

In [ ]:
angx=np.zeros([1,nT,nS])
mxdx=np.zeros([1,nT,nS])
clnx=np.zeros([1,nT,nS])
anix=np.zeros([1,nT,nS])
sghx=np.zeros([1,nT,nS])


for iT in np.arange(nT):
    angx[0,iT,:]=angs
    mxdx[0,iT,:]=mxds
    clnx[0,iT,:]=clns
    anix[0,iT,:]=anis
    sghx[0,iT,:]=sghs
    
angx = np.reshape( angx, (1,nT*nS ) )
mxdx = np.reshape( mxdx, (1,nT*nS ) )
clnx = np.reshape( clnx, (1,nT*nS ) )
anix = np.reshape( anix, (1,nT*nS ) )
sghx = np.reshape( sghx, (1,nT*nS ) )

In [ ]:
np.shape(tu)

In [ ]:
np.shape(angs)

In [ ]:
plt.plot(clns.flatten(),'.')

In [ ]:
print(np.shape(mxdx))
plt.plot(cln.view())
plt.plot(mxdx.values.flatten(),'.')

In [ ]:
np.mean(tu)

In [ ]:
cu = u
cu[np.where(np.isnan(cu))]=np.nanmean(cu) ### remove nan values



In [ ]:
cv = v
cv[np.where(np.isnan(cv))]=np.nanmean(cv) ### remove nan values


In [ ]:
ctu = tu
ctu[np.where(np.isnan(ctu))]=np.nanmean(ctu) 

In [ ]:
np.mean(tv)

In [ ]:
ctv = tv
ctv[np.where(np.isnan(ctv))]=np.nanmean(ctv) 

In [ ]:
plt.plot(ctv.flatten(),'.')

In [ ]:
np.mean(u)

In [ ]:
ctv.shape

In [ ]:
jj=np.where(np.isnan(u))

In [ ]:
print(jj)

In [ ]:
###### standardize each input variable ###########

scaler = StandardScaler()
scaler.fit(cu)
#print(scaler.mean_)
cu_scaled = scaler.transform(cu)  ### normalize dataset

#print(np.nanmin(u))

In [ ]:
np.mean(cu_scaled)

In [ ]:
scaler.fit(cv)
cv_scaled = scaler.transform(cv)  ### normalize dataset


In [ ]:
np.mean(cv_scaled)

In [ ]:
scaler.fit(ctu)
ctu_scaled = scaler.transform(ctu)  ### normalize dataset

scaler.fit(ctv)
ctv_scaled = scaler.transform(ctv)  ### normalize dataset



In [ ]:
plt.plot(ctv.flatten(),'.')

In [ ]:
np.min(ctv)

In [ ]:
def normalize(arr, t_min, t_max):
    norm_arr = []
    diff = t_max - t_min
    diff_arr = max(arr) - min(arr)   
    for i in arr:
        temp = (((i - min(arr))*diff)/diff_arr) + t_min
        norm_arr.append(temp)
    return norm_arr
 

In [ ]:
np.min(clnx)

In [ ]:
mxdx_scaled = (mxdx-np.min(mxdx))/np.ptp(mxdx)  ### normalize dataset

clnx_scaled = (clnx-np.min(clnx))/np.ptp(clnx)  ### normalize dataset

angx_scaled = (angx-np.min(angx))/np.ptp(angx)  ### normalize dataset

anix_scaled = (anix-np.min(anix))/np.ptp(anix)  ### normalize dataset

sghx_scaled = (sghx-np.min(sghx))/np.ptp(sghx)  ### normalize dataset


In [ ]:
np.max(sghx_scaled)

In [ ]:
x_total = np.r_[cu_scaled,cv_scaled,angx_scaled,mxdx_scaled,clnx_scaled,anix_scaled,sghx_scaled]
y_total = np.r_[ctu_scaled,ctv_scaled]


In [ ]:
x_total = np.r_[cu,cv,angx,mxdx,clnx,anix,sghx]
y_total = np.r_[ctu,ctv]


In [ ]:
y_total.shape

In [ ]:
print(y_total.shape)

In [ ]:
x_total = np.transpose(x_total)
y_total = np.transpose(y_total)


In [ ]:
print(y_total.shape)

In [ ]:
plt.plot(ctv_scaled.flatten(),'.')

In [ ]:
##x_total = np.stack((u,v,angx,mxdx,clnx,anix),1)
##y_total = np.stack((tu,tv),1)

In [ ]:
##### After data preprocessing, import tensorflow and keras build the CNN regression model, train and test ###########
normalizer = Normalization(axis=-1)
normalizer.adapt(x_total)
print(normalizer.mean.numpy())


In [ ]:
ctu_scaled[:,50000]

In [ ]:
plt.plot(angx_scaled.flatten(),'.')

In [ ]:
x_total.shape

In [ ]:
y_total.shape

In [ ]:
y_total[10000,:]

In [ ]:
x_total = x_total.reshape(x_total.shape[0], x_total.shape[1])  ### add the third dimension that will be the number of the single input row.
print(x_total.shape)

In [ ]:
y_total = y_total.reshape(y_total.shape[0], y_total.shape[1])
print(y_total.shape)

In [ ]:
na = np.arange(0.1,0.99,0.01)

In [ ]:
print(na)

In [ ]:
na[

In [ ]:
#scaler = StandardScaler()
#scaler.fit(TrainDataset)
#scaler.mean_,scaler.scale_
#x_train_scaled = scaler.transform(TrainDataset)  ### normalize dataset
#x_val_scaled = scaler.transform(ValDataset)

na = np.arange(0.1,0.99,0.01)
score_train = np.zeros(89)
score_eval = np.zeros(89)
kk = 0

for nn in na[20:21]:

    kk=kk+1
    x_train, x_test, y_train, y_test = train_test_split(x_total, y_total, test_size=nn, random_state=40) ## split training and test dataset
    n_inputs = 27
    n_outputs = 22
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu')) ### MLP
    model.add(Dense(n_outputs))
    model.compile(loss='mae', optimizer='adam')
#add model layers
#model.compile(optimizer='Adam',
#              loss='mse'), ##### 'mean_absolute_error')


    model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))

    score = model.evaluate(x_test, y_test, verbose=1)
    y_pred = model.predict(x_test)
    score2 = model.evaluate(x_train, y_train)
    
##    score_train[kk]=score
##    score_eval[kk]=score2
    
    



In [ ]:
plt.plot(na,score_train)
plt.plot(na,score_eval)

In [ ]:
x_test.shape

In [ ]:
x_train[10000,:]

In [ ]:
x_train.shape

In [ ]:
from sklearn.datasets import make_regression

In [ ]:
X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=3, random_state=2)


In [ ]:
y.shape

In [ ]:
#create model
n_inputs = 27
n_outputs = 22
model = Sequential()
model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu')) ### MLP
model.add(Dense(n_outputs))
model.compile(loss='mae', optimizer='adam')
#add model layers
##model.add(Conv2D(64, kernel_size=3, activation=’relu’, input_shape=(np.shape(x_train[1]),np.shape(x_train[2])),1))
##model.add(Conv2D(32, kernel_size=3, activation=’relu’))
#model.add(Conv1D(32, 2, activation="relu", input_shape=(27, 1)))
##model.add(Flatten())
##model.add(Dense(64, activation="relu"))
##model.add(Dense(1))
##model.compile(loss="mse", optimizer="adam")

##model.add(Flatten())
##model.add(Dense(10, activation=’softmax’))

##model = Sequential()
##model.add(Dense(512, activation='relu'))
##model.add(Dropout(0.5))
##model.add(Dense(256, activation='relu'))
##model.add(Dropout(0.25))
##model.add(Dense(10))


In [ ]:
model.summary()

In [ ]:
#create model
model = Sequential()
#add model layers
model.add(Conv2D(64, kernel_size=3, activation=’relu’, input_shape=(np.shape(x_train[1]),np.shape(x_train[2])),1))
model.add(Conv2D(32, kernel_size=3, activation=’relu’))
#model.add(Conv1D(32, 2, activation="relu", input_shape=(27, 1)))
model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dense(1))
##model.compile(loss="mse", optimizer="adam")
model.summary()

In [ ]:
model.compile(optimizer='Adam',
              loss='mse'), ##### 'mean_absolute_error')

In [ ]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))

In [ ]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score)

In [ ]:
y_pred = model.predict(x_test)
#print(model.evaluate(x_train, y_train))
#print("MSE: %.4f" % mean_squared_error(y_test, y_pred))


In [ ]:
y_pred.shape

In [ ]:
y_test.shape

In [ ]:
plt.scatter(y_pred.flatten(),y_test.flatten())

xx = np.linspace(-40,40,100)
plt.plot(xx,xx,'-r')


In [ ]:
model.compile(optimizer='Adam',
              loss='mse'), ##### 'mean_absolute_error')

model.summary()

In [ ]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))


In [ ]:
y_train.shape

In [ ]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])




In [ ]:
y_pred = model.predict(x_test)
print(model.evaluate(x_train, y_train))
#print("MSE: %.4f" % mean_squared_error(y_test, y_pred))


In [ ]:
x_test.shape

In [ ]:
x_ax = range(len(ypred))
plt.scatter(x_ax, ytest, s=5, color="blue", label="original")
plt.plot(x_ax, ypred, lw=0.8, color="red", label="predicted")
plt.legend()
plt.show()